In [1]:
import pycocotools.coco as coco
import json
import os
import os.path as osp
from PIL import Image
from tqdm import tqdm
import numpy as np
import shutil
# data = np.array(data)

In [2]:
data_path = '/media/allen/mass/DB'
branch = 'prw'

src = osp.join(data_path, branch)
train_dst = osp.join(src, 'train2017')
val_dst = osp.join(src, 'val2017')
if not osp.exists(train_dst):
    os.mkdir(train_dst)
if not osp.exists(val_dst):
    os.mkdir(val_dst)
label_src = osp.join(src, "labels_with_ids")
# img_list = "/media/allen/mass/Towards-Realtime-MOT/data/prw.val"
# img_paths = []
# with open(img_list, 'r') as f:
#     for line in f.readlines() :
#         line = line.strip()
#         img_paths.append(osp.join(data_path, line))
        # break
# img_paths = [osp.join(root, f) for root, _, files in os.walk(src) for f in files if '.jpg' in f or '.png' in f]

In [3]:
img_paths[0]

NameError: name 'img_paths' is not defined

In [3]:
dataset = {
    "info": {},
    "licenses": [],
    "images": [],
    "annotations": [],
    "categories": []
}
dataset['categories'].append({
    'id': 1,
    'name': "person",
    'supercategory': "people",
    'keypoints': [],
    'skeleton': []
})

anno_paths = [osp.join(root, f) for root, _, files in os.walk(label_src) for f in files if '.txt' in f]

sub_index = 0
idx = 0
for anns in tqdm(anno_paths):
    path = osp.join(src, "images", f"{osp.splitext(osp.basename(anns))[0]}.jpg")
    dst = osp.join(train_dst, osp.basename(path))
    img = Image.open(path)
    w, h = img.size
    idx += 1
    dataset['images'].append({
        'coco_url': '',
        'date_captured': '',
        'file_name': osp.basename(dst),
        'flickr_url': '',
        'id': idx,
        'license': 0,
        'width': w,
        'height': h
    })
    with open(anns, 'r') as f:
        for line in f.readlines():
            sub_index += 1
            _class, identity, x_center, y_center, width, height = line.strip().replace("  ", " ").split(" ")
            x1 = (float(x_center) - float(width)/2) * w
            y1 = (float(y_center) - float(height)/2) * h
            width = float(width) * w
            height = float(height) * h
            dataset['annotations'].append({
                'area': int(width*height),
                'bbox': [int(x1),int(y1),int(width),int(height)],
                'category_id': 1,
                'pid': identity,
                'id': sub_index,
                'image_id': idx,
                'iscrowd': 0,
                'segmentation': [],
            })
    # shutil.copy(path, osp.join(train_dst, osp.basename(path)))
    # break

100%|██████████| 11792/11792 [04:28<00:00, 42.97it/s]


In [4]:
with open(osp.join(src, 'instances_train2017.json'), 'w') as f:
    json.dump(dataset, f)

In [9]:
with open(osp.join(src, 'instances_val2017.json'), 'w') as f:
    json.dump(dataset, f)

In [10]:
import cv2

In [15]:
for bb in l_bbs:
    x1, y1, w, h = bb
    cv2.rectangle(l_img, (int(x1), int(y1)), (int(x1+w), int(y1+h)), (255,255,255), 2)

In [5]:
co = coco.COCO(osp.join(src, 'instances_train2017.json'))

loading annotations into memory...
Done (t=0.28s)
creating index...
index created!


In [6]:
image_ids = co.getImgIds()
img_id = image_ids[150]
idxs = co.getAnnIds(imgIds=[img_id])
fname = co.loadImgs(ids=[img_id])[0]['file_name']

In [7]:
fname

'PRW/train2017/c1s1_046476.jpg'

In [8]:
img = Image.open(os.path.join(data_path, fname))
anns = co.loadAnns(idxs)

In [12]:
np_img = np.array(img)
for ann in anns:
    x1, y1, w, h = ann['bbox']
    cv2.rectangle(np_img, (int(x1), int(y1)), (int(x1+w), int(y1+h)), (255,255,255), 2)
Image.fromarray(np_img)